In [46]:
import sys
import uproot

import os
import numpy as np
import pandas as pd

In [47]:
part = "gamma" # "muon" "electron" "pion_c"
path = "/data/user/adipilat/ParticleID/genEvts/"
unpad_path = "/data/user/adipilat/ParticleID/genEvts/new_datasets/unpadded/"
pad_path = "/data/user/adipilat/ParticleID/genEvts/new_datasets/padded/"
dir_ = "ana"
tree = "hgc"
max_perlayer = 10
number_layers = 50

In [48]:
variableName = [
            'event',
            'cluster2d_layer',
            'cluster2d_energy',
            'cluster2d_eta',
            'cluster2d_phi',
            'cluster2d_pt',
            'cluster2d_x',
            'cluster2d_y',
            'cluster2d_z',
            'cluster2d_nhitCore',
            'cluster2d_nhitAll',
            'gen_energy',
            'gen_pdgid',
            'gen_daughters',
            'gen_phi',
            'gen_eta',
            'trackster_clusters'
            ]
newVars =["event","trackster","x","y","z","r","layer","E","nCore","nHits","id","genDR","gen_phi","gen_eta","phi","eta","pid","genE"]

In [49]:
name = "4_" + part
file = path + part + "/" + name + ".root"
print("Starting data production for "+ part)

Starting data production for gamma


In [50]:
df = uproot.open(file)[dir_][tree].pandas.df(variableName,flatten=False)


num_events = np.unique(df["event"].values).shape[0]
xs = df["cluster2d_x"].values
ys = df["cluster2d_y"].values
zs = df["cluster2d_z"].values
es = df["cluster2d_energy"].values
ps = df["cluster2d_pt"].values
nh = df["cluster2d_nhitAll"].values
nc = df["cluster2d_nhitCore"].values
ll = df["cluster2d_layer"].values
ee = df["event"].values
    
sizes = [x.shape[0] for x in xs]
indices = [np.full((a[0]),a[1]) for a in zip(sizes,range(len(sizes)))]

cphi = df["cluster2d_phi"].values
ceta = df["cluster2d_eta"].values
gpid = df["gen_pdgid"].values
gen = df["gen_energy"].values

gphi = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_phi"].values)]
geta = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_eta"].values)]
gpid = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_pdgid"].values)]
gen = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_energy"].values)]


tr = df["trackster_clusters"].values

In [51]:
idtrlist = []
# LayerClusters that don't belong to any Trackster will have TracksterId = 0. Real Tracksters have the TracksterId > 0
for i in range(len(sizes)):
    idtrlist.append(np.array([0]*sizes[i]))
for i in range(len(tr)):
    for j in range(len(tr[i])):
        for item in tr[i][j]:
            idtrlist[i][item] = j + 1

idtr = np.array(idtrlist)
print(idtr.shape)

(10000,)


In [52]:
rs = [np.sqrt(f[0]**2+f[1]**2) for f in zip(xs,ys)]
drs = [np.sqrt((a[0]-a[1])**2 + (a[2]-a[3])**2) for a in zip(gphi,cphi,geta,ceta)]

In [53]:
XS = np.array([item for sublist in xs for item in sublist])
YS = np.array([item for sublist in ys for item in sublist])
ZS = np.array([item for sublist in zs for item in sublist])
RS = np.array([item for sublist in rs for item in sublist])
LL = np.array([item for sublist in ll for item in sublist])
ES = np.array([item for sublist in es for item in sublist])
NC = np.array([item for sublist in nc for item in sublist])
NH = np.array([item for sublist in nh for item in sublist])
II = np.array([item for sublist in indices for item in sublist])
DRS = np.array([item for sublist in drs for item in sublist])
GPHI = np.array([item for sublist in gphi for item in sublist])
GETA = np.array([item for sublist in geta for item in sublist])
GPID = np.array([item for sublist in gpid for item in sublist])
GEN = np.array([item for sublist in gen for item in sublist])
CPHI = np.array([item for sublist in cphi for item in sublist])
CETA = np.array([item for sublist in ceta for item in sublist])


SS = [np.full((s,),s) for s in sizes]
EE = [np.full((s,),i) for i,s in zip(ee,sizes)]

SS = np.array([item for sublist in SS for item in sublist])
EE = np.array([item for sublist in EE for item in sublist])

TR = np.array([item for sublist in idtr for item in sublist])

datas = np.vstack((EE,TR,XS,YS,ZS,RS,LL,ES,NC,NH,II,DRS,GPHI,GETA,CPHI,CETA,GPID,GEN)).T

In [54]:
df = pd.DataFrame(datas,columns=newVars)
df = df.sort_values(["event","trackster","layer","E"],ascending=[True,True,True,False]).reset_index(drop=True)

In [55]:
df = df[df["trackster"]!=0].reset_index(drop=True)
df.head()

,event,trackster,x,y,z,r,layer,E,nCore,nHits,id,genDR,gen_phi,gen_eta,phi,eta,pid,genE
0,1.0,1.0,63.384552,-82.981453,325.072754,104.419930,3.0,0.019288,2.0,2.0,3.0,0.096535,-0.890417,1.945988,-0.918496,1.853627,22.0,62.874428
1,1.0,1.0,58.605431,-81.999992,325.072754,100.789856,3.0,0.015269,1.0,1.0,3.0,0.083779,-0.890417,1.945988,-0.950272,1.887369,22.0,62.874428
2,1.0,1.0,49.136902,-63.183201,328.042755,80.040939,5.0,0.026119,1.0,1.0,3.0,0.173415,-0.890417,1.945988,-0.909808,2.118315,22.0,62.874428
3,1.0,1.0,49.086895,-72.937004,328.987274,87.916611,6.0,0.016475,1.0,1.0,3.0,0.121784,-0.890417,1.945988,-0.978419,2.030171,22.0,62.874428
4,1.0,1.0,61.438732,-76.029900,331.012726,97.751030,7.0,6.109771,17.0,17.0,3.0,0.012008,-0.890417,1.945988,-0.891144,1.934002,22.0,62.874428


In [56]:
df.to_hdf(unpad_path + part + "_new.h5","data",complevel=0)

In [57]:
trackster_sizes = df.groupby(["event","trackster"]).size().values.tolist()
trackster_places = np.cumsum(trackster_sizes)
num_tracksters = len(trackster_sizes)
track_startes = np.array( [0] + list(trackster_places[:-1]))
track_finishes = np.array(list(track_startes[1:]) +[len(df)])
track_id = np.arange(1,num_tracksters+1)
track_bounds = np.vstack((track_startes,track_finishes)).T

In [58]:
new_tracks = [[i for j in range(t[1]-t[0])] for i,t in zip(track_id, track_bounds)]
new_tracks = np.array([item for sublist in new_tracks for item in sublist])
df['trackster'] = new_tracks
df.head()

,event,trackster,x,y,z,r,layer,E,nCore,nHits,id,genDR,gen_phi,gen_eta,phi,eta,pid,genE
0,1.0,1,63.384552,-82.981453,325.072754,104.419930,3.0,0.019288,2.0,2.0,3.0,0.096535,-0.890417,1.945988,-0.918496,1.853627,22.0,62.874428
1,1.0,1,58.605431,-81.999992,325.072754,100.789856,3.0,0.015269,1.0,1.0,3.0,0.083779,-0.890417,1.945988,-0.950272,1.887369,22.0,62.874428
2,1.0,1,49.136902,-63.183201,328.042755,80.040939,5.0,0.026119,1.0,1.0,3.0,0.173415,-0.890417,1.945988,-0.909808,2.118315,22.0,62.874428
3,1.0,1,49.086895,-72.937004,328.987274,87.916611,6.0,0.016475,1.0,1.0,3.0,0.121784,-0.890417,1.945988,-0.978419,2.030171,22.0,62.874428
4,1.0,1,61.438732,-76.029900,331.012726,97.751030,7.0,6.109771,17.0,17.0,3.0,0.012008,-0.890417,1.945988,-0.891144,1.934002,22.0,62.874428


In [59]:
# get rid of useless stuff
del df['id']
del df['event']
df.head()

,trackster,x,y,z,r,layer,E,nCore,nHits,genDR,gen_phi,gen_eta,phi,eta,pid,genE
0,1,63.384552,-82.981453,325.072754,104.419930,3.0,0.019288,2.0,2.0,0.096535,-0.890417,1.945988,-0.918496,1.853627,22.0,62.874428
1,1,58.605431,-81.999992,325.072754,100.789856,3.0,0.015269,1.0,1.0,0.083779,-0.890417,1.945988,-0.950272,1.887369,22.0,62.874428
2,1,49.136902,-63.183201,328.042755,80.040939,5.0,0.026119,1.0,1.0,0.173415,-0.890417,1.945988,-0.909808,2.118315,22.0,62.874428
3,1,49.086895,-72.937004,328.987274,87.916611,6.0,0.016475,1.0,1.0,0.121784,-0.890417,1.945988,-0.978419,2.030171,22.0,62.874428
4,1,61.438732,-76.029900,331.012726,97.751030,7.0,6.109771,17.0,17.0,0.012008,-0.890417,1.945988,-0.891144,1.934002,22.0,62.874428


In [60]:
theIndex = list(df.groupby(["trackster","layer"]).indices.values())
theIndex = np.array([item for sublist in theIndex for item in sublist[:min(len(sublist),10)]])

In [61]:
df = df.iloc[theIndex]
df.head()

,trackster,x,y,z,r,layer,E,nCore,nHits,genDR,gen_phi,gen_eta,phi,eta,pid,genE
0,1,63.384552,-82.981453,325.072754,104.419930,3.0,0.019288,2.0,2.0,0.096535,-0.890417,1.945988,-0.918496,1.853627,22.0,62.874428
1,1,58.605431,-81.999992,325.072754,100.789856,3.0,0.015269,1.0,1.0,0.083779,-0.890417,1.945988,-0.950272,1.887369,22.0,62.874428
2,1,49.136902,-63.183201,328.042755,80.040939,5.0,0.026119,1.0,1.0,0.173415,-0.890417,1.945988,-0.909808,2.118315,22.0,62.874428
3,1,49.086895,-72.937004,328.987274,87.916611,6.0,0.016475,1.0,1.0,0.121784,-0.890417,1.945988,-0.978419,2.030171,22.0,62.874428
4,1,61.438732,-76.029900,331.012726,97.751030,7.0,6.109771,17.0,17.0,0.012008,-0.890417,1.945988,-0.891144,1.934002,22.0,62.874428


In [62]:
layer_sizes = df.groupby(["trackster","layer"]).size().values.tolist()
layer_places = np.cumsum(layer_sizes)

In [63]:
startes = np.array( [0] + list(layer_places[:-1]))
layers = df["layer"].values[startes]
ids = df["trackster"].values[startes]
finishes = np.array(list(startes[1:]) +[len(df)])
SSS = np.vstack((startes,finishes)).T

In [64]:
hitIds = [[j +(n-1)*max_perlayer + max_perlayer*number_layers*(e-1) for j in range(s[1]-s[0])] for n,s,e in zip(layers,SSS,ids)]
hitIds = np.array([item for sublist in hitIds for item in sublist])

In [65]:
df.loc[:,"hitIds"] = hitIds
df = df.set_index(hitIds.astype(int))

In [66]:
bigMask = np.zeros((num_tracksters*number_layers*max_perlayer,len(df.columns)))
bigDF = pd.DataFrame(bigMask,columns=df.columns)

In [67]:
fakeHit = [ [(i*max_perlayer + j) for j in range(max_perlayer)] for i in range(number_layers*num_tracksters)]
fakeHit = np.array([item for sublist in fakeHit for item in sublist])

In [68]:
fakeLayer = [ np.full(max_perlayer,i) for j in range(1,num_tracksters+1) for i in range(1,number_layers+1)]
fakeLayer = np.array([item for sublist in fakeLayer for item in sublist])

In [69]:
fakeTrackster = [ np.full(max_perlayer*number_layers,i) for i in range(1,num_tracksters+1)]
fakeTrackster = np.array([item for sublist in fakeTrackster for item in sublist])

In [70]:
bigDF["layer"] = fakeLayer
bigDF["trackster"] = fakeTrackster
bigDF["hitIds"] = fakeHit

In [71]:
bigDF.iloc[df.index] = df

In [72]:
bigDF[0:50]

,trackster,x,y,z,r,layer,E,nCore,nHits,genDR,gen_phi,gen_eta,phi,eta,pid,genE,hitIds
0,1,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
1,1,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.0
2,1,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,2.0
3,1,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,3.0
4,1,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,4.0
5,1,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,5.0
6,1,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,6.0
7,1,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,7.0
8,1,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,8.0
9,1,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,9.0


In [73]:
del bigDF['hitIds']
bigDF.head()

,trackster,x,y,z,r,layer,E,nCore,nHits,genDR,gen_phi,gen_eta,phi,eta,pid,genE
0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
bigDF.to_hdf(pad_path + part + "_newPadded.h5","data",complevel=0)